In [ ]:
import tensorflow as tf

def flip_left_right(image, label):
    image = tf.image.flip_left_right(image)
    return image, label

def flip_up_down(image, label):
    image = tf.image.flip_up_down(image)
    return image, label

def rot(image, label, k_value):
    image = tf.image.rot90(image, k=k_value)
    return image, label

def rot90(image, label):
    return rot(image, label, 1)

def rot180(image, label):
    return rot(image, label, 2)

def rot270(image, label):
    return rot(image, label, 3)

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "dog_cat_photos/train",
    image_size=(75, 75),
    label_mode="binary",
    batch_size=32,
    shuffle=False
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "dog_cat_photos/test",
    image_size=(75, 75),
    label_mode="binary",
    batch_size=32,
    shuffle=False
)

train_dataset_lr     = train_dataset.map(flip_left_right)
train_dataset_ud     = train_dataset.map(flip_up_down)
train_dataset_rot90  = train_dataset.map(rot90)
train_dataset_rot180 = train_dataset.map(rot180)
train_dataset_rot270 = train_dataset.map(rot270)

train_dataset = train_dataset.concatenate(train_dataset_lr)
train_dataset = train_dataset.concatenate(train_dataset_ud)
train_dataset = train_dataset.concatenate(train_dataset_rot90)
train_dataset = train_dataset.concatenate(train_dataset_rot180)
train_dataset = train_dataset.concatenate(train_dataset_rot270)

train_dataset = train_dataset.shuffle(32)

input_layer = tf.keras.Input(shape=(75, 75, 3))
l_layer = tf.keras.applications.mobilenet_v2.preprocess_input(input_layer)

base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(75, 75, 3),
    input_tensor=l_layer,
    include_top=False,
    weights="imagenet",
    # 環境によって CERTIFICATE_VERIFY_FAILED が出るので、そのときはローカルにライブラリをgetしてきて指定して使う
    # weights="/tmp/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5",
    pooling='avg'
)
base_model.trainable = False
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([
    base_model,
    output_layer
])
model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=["accuracy"])
model.fit(train_dataset, epochs=20)
pred_data = model.predict(test_dataset)
model.evaluate(test_dataset)

Found 300 files belonging to 2 classes.
Found 100 files belonging to 2 classes.


/var/folders/76/t4t43jd15dx56jvsbv9pln3r0000gq/T/ipykernel_99324/318977543.py:57: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(


Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1010)